<a href="https://colab.research.google.com/github/harryduffy/ResNet/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import sys
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers.legacy import RMSprop
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from keras.layers import Input, Conv2D, UpSampling2D, BatchNormalization, Activation, add, concatenate
from tensorflow.keras import layers, models
import random
import matplotlib.pyplot as plt
import cv2

sys.path.append('/content/drive/MyDrive/Road Extraction/Duffy/ResNet/')
drive.mount('/content/drive')

from util import load_data, preprocess_data
# from res_unet import build_res_unet

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def residual_block(x, filters):
    shortcut = layers.Conv2D(filters, (1, 1), padding='same')(x)
    shortcut = layers.BatchNormalization()(shortcut)

    x = layers.Conv2D(filters, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(filters, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)

    # Add the shortcut to the output of the convolution block
    x = layers.add([x, shortcut])
    x = layers.Activation('relu')(x)
    return x

def encoder_block(x, filters):
    x = residual_block(x, filters)
    p = layers.MaxPooling2D((2, 2))(x)
    return x, p

def decoder_block(x, skip, filters):
    x = layers.Conv2DTranspose(filters, (2, 2), strides=(2, 2), padding='same')(x)
    x = layers.concatenate([x, skip])
    x = residual_block(x, filters)
    return x

def build_res_unet(input_shape):
    inputs = layers.Input(shape=input_shape)

    # Encoder
    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    # Bridge
    b1 = residual_block(p4, 1024)

    # Decoder
    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(d4)

    model = models.Model(inputs, outputs)
    return model

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    print('GPU is available:', physical_devices)
    try:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
        print('Memory growth set for GPU.')
    except Exception as e:
        print(f'Error setting memory growth: {e}')
else:
    print('GPU is not available')


# initialize input size and shape
INPUT_SIZE = (256, 256)
INPUT_SHAPE = (256, 256, 3) # color images, 3 channels

def display_data(dir_path, image_paths, mask_paths):

    fig, axes = plt.subplots(5, 2, figsize=(10, 15))

    # Iterate over the image and mask pairs and display them in subplots
    for i, (image_path, mask_path) in enumerate(zip(image_paths, mask_paths)):
        # Load the image and mask using your preferred method
        image = plt.imread(dir_path + image_path)
        mask = plt.imread(dir_path + mask_path)

        # Plot the image and mask in the corresponding subplot
        axes[i, 0].imshow(image)
        axes[i, 0].set_title('Image')
        axes[i, 0].axis('off')

        axes[i, 1].imshow(mask)
        axes[i, 1].set_title('Mask')
        axes[i, 1].axis('off')

    # Adjust the spacing between subplots
    plt.tight_layout()
    plt.savefig('samples.png', bbox_inches='tight')  # Save as PNG image

    # Show the plot
    plt.show()
    return

# load data

dir_path = '/content/drive/MyDrive/Road Extraction/Duffy/tiles/'
image_filenames, mask_filenames = load_data(dir_path)

print(len(image_filenames))

# display the first 5 pairs of image and mask
# random_indices = random.choices(range(0, len(image_filenames)), k=5)
# display_data(dir_path, image_filenames[random_indices], mask_filenames[random_indices])

GPU is available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Memory growth set for GPU.
3236


In [ ]:
# preprocess data (BBBRRRRRRT)
with tf.device("/device:GPU:0"):
    images, masks = preprocess_data(dir_path, image_filenames, mask_filenames, input_size=INPUT_SIZE, augmented=False)

In [ ]:
# get shape of the image and mask
print('Shape of image data: ' + str(images.shape))
print('Shape of mask data: ' + str(masks.shape))

train_images, val_images, train_masks, val_masks = train_test_split(images, masks, test_size=0.2, random_state=42)
train_images, test_images, train_masks, test_masks = train_test_split(train_images, train_masks, test_size=0.2, random_state=42)

Shape of image data: (3236, 256, 256, 3)
Shape of mask data: (3236, 256, 256, 1)


In [ ]:
# initialise the model and get a summary
model = build_res_unet(INPUT_SHAPE)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv2d_29 (Conv2D)          (None, 256, 256, 64)         1792      ['input_5[0][0]']             
                                                                                                  
 batch_normalization_27 (Ba  (None, 256, 256, 64)         256       ['conv2d_29[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_16 (Activation)  (None, 256, 256, 64)         0         ['batch_normalization_27[0

In [ ]:
# Import necessary libraries
from tensorflow.keras import backend as K

# Define functions for evaluation metrics
def dice_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
    dice = K.mean((2. * intersection + smooth) / (union + smooth), axis=0)
    return dice

def iou(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
    union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3]) - intersection
    iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
    return iou

def f1_score(y_true, y_pred):
    precision = K.sum(K.round(K.clip(y_true * y_pred, 0, 1))) / (K.sum(K.round(K.clip(y_pred, 0, 1))) + K.epsilon())
    recall = K.sum(K.round(K.clip(y_true * y_pred, 0, 1))) / (K.sum(K.round(K.clip(y_true, 0, 1))) + K.epsilon())
    f1 = 2 * (precision * recall) / (precision + recall + K.epsilon())
    return f1

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('models/save_best.h5', verbose=1, save_best_only=True)

# Compile model with custom metrics
learning_rate = 0.0001
model.compile(optimizer=RMSprop(learning_rate=learning_rate), loss='binary_crossentropy', metrics=['accuracy', dice_coef, iou, f1_score])

# Train the model while monitoring custom metrics
epochs = 25
history = model.fit(train_images, train_masks, batch_size=16, epochs=epochs, validation_data=(val_images, val_masks),callbacks=[checkpoint])

Epoch 1/100
130/130 [==============================] - ETA: 0s - loss: 0.2619 - accuracy: 0.9245 - dice_coef: 0.0545 - iou: 0.0287 - f1_score: 0.0123
Epoch 1: val_loss improved from inf to 0.20153, saving model to models/save_best.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


130/130 [==============================] - 251s 2s/step - loss: 0.2619 - accuracy: 0.9245 - dice_coef: 0.0545 - iou: 0.0287 - f1_score: 0.0123 - val_loss: 0.2015 - val_accuracy: 0.9574 - val_dice_coef: 0.0516 - val_iou: 0.0270 - val_f1_score: 4.1439e-06
Epoch 2/100
130/130 [==============================] - ETA: 0s - loss: 0.1768 - accuracy: 0.9553 - dice_coef: 0.0419 - iou: 0.0218 - f1_score: 0.0000e+00
Epoch 2: val_loss improved from 0.20153 to 0.17639, saving model to models/save_best.h5
130/130 [==============================] - 154s 1s/step - loss: 0.1768 - accuracy: 0.9553 - dice_coef: 0.0419 - iou: 0.0218 - f1_score: 0.0000e+00 - val_loss: 0.1764 - val_accuracy: 0.9574 - val_dice_coef: 0.0395 - val_iou: 0.0205 - val_f1_score: 0.0000e+00
Epoch 3/100
130/130 [==============================] - ETA: 0s - loss: 0.1743 - accuracy: 0.9553 - dice_coef: 0.0424 - iou: 0.0221 - f1_score: 0.0000e+00
Epoch 3: val_loss did not improve from 0.17639
130/130 [==============================] - 14

KeyboardInterrupt: 

In [ ]:
# Accessing training and testing accuracy
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

epochs = range(1, epochs + 1)

# Plotting
plt.plot(epochs, train_accuracy, 'r', label='Training Accuracy')
plt.plot(epochs, val_accuracy, 'b', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# Assuming 'history' object contains the IoU values at each epoch
iou_values = history.history['iou']
val_iou_values = history.history['val_iou']

# Number of epochs
epochs = range(1, len(iou_values) + 1)

# Plotting the line plot
plt.figure(figsize=(8, 6))
plt.plot(epochs, iou_values, marker='o', linestyle='-', color='r')
plt.plot(epochs, val_iou_values, marker='o', linestyle='-', color='b')
plt.title('IoU and val_iou across Epochs')
plt.xlabel('Number of Epochs')
plt.ylabel('IoU Value')
plt.grid(True)
plt.show()